In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rnd
from scipy.stats import mannwhitneyu

In [ ]:
df = pd.read_csv('new_dataframe.csv', sep = ',')
df.head()

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
0,0,867689,12,iOS,business,no surge,1,1,1,1,0,0,7.982135,20,low
1,1,752172,5,Android,economy,no surge,1,1,1,1,0,1,2.908468,27,high
2,2,486559,15,Android,comfort,no surge,1,1,1,1,0,0,7.224614,21,high
3,3,304024,0,Android,economy,no surge,1,1,1,1,0,1,1.874349,52,low
4,4,139420,0,Android,business,no surge,1,1,1,1,0,0,10.704778,19,low


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101500 entries, 0 to 101499
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         101500 non-null  int64  
 1   user_id            101500 non-null  int64  
 2   hour               101500 non-null  int64  
 3   os                 101500 non-null  object 
 4   order_class        101500 non-null  object 
 5   surge              91431 non-null   object 
 6   app_opened         101500 non-null  int64  
 7   price_seen         101500 non-null  int64  
 8   order_made         101500 non-null  int64  
 9   ride_completed     101500 non-null  int64  
 10  user_cancelled     101500 non-null  int64  
 11  city_center_order  101500 non-null  int64  
 12  distance           91431 non-null   float64
 13  age                101500 non-null  int64  
 14  rfm                101500 non-null  object 
dtypes: float64(1), int64(10), object(4)
memory usage: 1

In [ ]:
#создаем датафрейм по заказам, совершенным в период Surge, где была показана цена
df_surge_ps = df[(df['surge'] == 'surge') & (df['price_seen'] == 1)]
df_surge_ps.head()

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
9,9,662775,17,Android,comfort,surge,1,1,1,1,0,1,4.828326,36,low
10,10,464055,20,iOS,comfort,surge,1,1,1,1,0,0,3.647097,25,high
18,18,393096,23,Android,comfort,surge,1,1,0,0,0,1,2.523357,19,high
21,21,103860,13,iOS,economy,surge,1,1,1,1,0,1,7.934002,21,high
24,24,461185,21,iOS,economy,surge,1,1,1,1,0,1,1.590209,29,high


In [ ]:
#Считаем количество совершенных заказов
df_surge_ps['order_made'].sum()

22873

In [ ]:
#Считаем % совершенных заказов от общего количества заказов, где была показана цена
print('Количество совершенных заказов составляет', (df_surge_ps['order_made'].sum()*100 / df_surge_ps['price_seen'].sum()).round(), '% от общего количества заказов, где была показана цена')

Количество совершенных заказов составляет 72.0 % от общего количества заказов, где была показана цена


In [ ]:
#посчитаем среднюю конверсию нажатия кнопки совершения заказа пользователями, которые увидели стоимость поездки в период высокого спроса
df_mean_conv = df_surge_ps.groupby(['user_id'])['order_made'].mean().reset_index()
df_mean_conv.head()

,user_id,order_made
0,100093,0.5
1,100118,1.0
2,100134,1.0
3,100166,1.0
4,100255,0.0


In [ ]:
#произведем рандомную разбивку пользователей на 2 группы
df_mean_conv['random_value'] = np.random.uniform(0,1, size = len(df_mean_conv['user_id']))

In [ ]:
def groups_gen(row):
  if row['random_value'] <=0.5:
    return(1)
  else: 
    return(2)

In [ ]:
df_mean_conv['group'] = df_mean_conv.apply(groups_gen, axis=1)
df_mean_conv.head()

,user_id,order_made,random_value,group
0,100093,0.5,0.069799,1
1,100118,1.0,0.868554,2
2,100134,1.0,0.375783,1
3,100166,1.0,0.972345,2
4,100255,0.0,0.655782,2


In [ ]:
df_mean_conv_g1 = df_mean_conv[df_mean_conv['group'] == 1]
df_mean_conv_g2 = df_mean_conv[df_mean_conv['group'] == 2]

In [ ]:
# проверим размеры выборок и выровняем их
len(df_mean_conv_g1['user_id'])

8998

In [ ]:
len(df_mean_conv_g2['user_id'])

8782

In [ ]:
# первая группа имеет больше пользователей. Сократим их количество, чтобы оно было равно количеству пользователей во 2-й группе
group_volume = min(len(df_mean_conv_g1['user_id']), len(df_mean_conv_g2['user_id']))
df_mean_conv_g1 = df_mean_conv_g1.iloc[0:group_volume-1]
df_mean_conv_g2 = df_mean_conv_g2.iloc[0:group_volume-1]

In [ ]:
len(df_mean_conv_g1['user_id'])

8781

In [ ]:
len(df_mean_conv_g2['user_id'])

8781

# Проверяем группы на однородность с помощью рангового критерия Манна-Уитни

Формируем H0 и H1:\
H0 Группы 1 и 2 являются однородными \
H1 Группы 1 и 2 имеют статистически значимые различия

In [ ]:
alpha = 0.05
stat, p = mannwhitneyu(df_mean_conv_g1['order_made'], df_mean_conv_g2['order_made'])
print('Statistics=%.3f, p=%.3f' % (stat, p))
if p > alpha:
  print('Группы 1 и 2 являются однородными (не отвергаем H0)')
else:
  print('Группы 1 и 2 имеют статистически значимые различия (отвергаем H0)')

Statistics=38720738.000, p=0.574
Группы 1 и 2 являются однородными (не отвергаем H0)
